In [1]:
from utils import load_model, load_config
import cv2
import torch

In [2]:
config = load_config('./configs/config_YOLO_x.yaml')
model = load_model(config['model'])

In [3]:
image = cv2.imread('./example_images/demo_cropped.jpg')
image = cv2.resize(image, config['dataset']['preprocess']['input_size'])
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = image.astype('float32')
image /= 255.0

image = torch.from_numpy(image).permute(2, 0, 1).unsqueeze(0)
print(image.shape)

torch.Size([1, 3, 640, 640])


In [13]:
model.train()
output = model(image)
print(output[1].shape)

AttributeError: 'list' object has no attribute 'shape'